In [363]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests
import time

In [364]:
custom_headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Accept-Language": "en-US,en;q=0.9", 
    "Upgrade-Insecure-Requests": "1", 
    "Referer": "https://www.google.com/",
    "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:126.0) Gecko/20100101 Firefox/126.0",
}

In [365]:
def get_soup(url):
    response = requests.get(url, headers=custom_headers)

    if response.status_code != 200:
        print("Error in getting webpage")
        exit(-1)

    soup = BeautifulSoup(response.text, "lxml")
    return soup

In [366]:
def get_reviews(reviewUrl):

    soup = get_soup(reviewUrl)
    review_elements = soup.select("div.review")

    scraped_reviews = []

    for item in review_elements:

        author = item.find('span', {'class': 'a-profile-name'}).text.strip() if item.find('span', {'class': 'a-profile-name'}) else ''

        rating = item.find('i', {'data-hook': 'review-star-rating'}).text.strip()[0] if item.find('i', {'data-hook': 'review-star-rating'}) else ''

        title = item.find('a', {'data-hook': 'review-title'}).text.strip().split('\n', 1)[1] if item.find('a', {'data-hook': 'review-title'}) else ''

        content = item.find('span', {'data-hook': 'review-body'}).text.strip() if item.find('span', {'data-hook': 'review-body'}) else ''

        date = item.find('span', {'data-hook': 'review-date'}).text.strip().split('on ', 1)[1]  if item.find('span', {'data-hook': 'review-date'}) else ''

        
        r = {
            "author": author,
            "rating": rating,
            "title": title,
            "content": content,
            "date": date
        }

        scraped_reviews.append(r)

    return scraped_reviews

In [367]:
def totalPages(productUrl):
    soup = get_soup(productUrl)
    reviews = soup.find('div', {'data-hook':"cr-filter-info-review-rating-count"})
    print(reviews.text.strip().split(', ')[1].split(" ")[0])
    return int(reviews.text.strip().split(', ')[1].split(" ")[0].replace(',',''))

In [368]:
iter_reviews = []
search_url = "https://www.amazon.co.uk/Yaheetech-Dining-Kitchen-Breakfast-Stools/product-reviews/B0C167TLF8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&"
reviewUrl = search_url + "pageNumber=" + str(3)
print(reviewUrl)
totalPage = totalPages(search_url)

for i in range(1):
    print(f"Running for page {i+1}")
    time.sleep(3)
    try:
        reviewUrl = search_url + "pageNumber=" + str(i+1)
        print(reviewUrl)
        data = get_reviews(reviewUrl)
    except Exception as e:
        print(e)
        
   
df = pd.DataFrame(data)
print(df)
df.to_csv("xboxControllerReview.csv")

https://www.amazon.co.uk/Yaheetech-Dining-Kitchen-Breakfast-Stools/product-reviews/B0C167TLF8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
56
Running for page 1
https://www.amazon.co.uk/Yaheetech-Dining-Kitchen-Breakfast-Stools/product-reviews/B0C167TLF8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
            author rating                            title  \
0      Sarah white      5  Great quality for a great price   
1             Tina      5      Sturdy and easy to assemble   
2               KC      4           Brilliant little table   
3      morganmarie      5                            great   
4  marguerite Bone      5          Fabulous table & stools   
5        Nickipo76      5    Great build quality and value   
6              Meh      5                          Amazing   
7       Cheee of E      4                         Stickers   
8    Jade Williams      4                Looks really good   
9            donna    